In [178]:
import bs4
import requests

In [179]:
def get_string(c):
    return ''.join(i.string if i.string is not None else "" for i in c.children)

def convert(section):
    children = list(section.children)
    if len(children) < 2:
        return

    # print(section.attrs)
    # title = children[1].string
    # if title is None or len(title.split()) > 1:
    #     return

    title = section.attrs['id'].split('/')[-1]

    # print(title)

    code_type = ""
    enum_vals = {}
    fields = {}

    for i in children[2:]:
        if i.name != 'section':
            continue

        if 'description' in i.attrs['id']:
            p = list(i.children)[1]
            description = get_string(p)
        elif 'ENUM_VALUES' in i.attrs['id']:
            code_type = "enum"
            for row in i.find("tbody").find_all("tr"):
                cs = list(row.children)
                td1, td2 = cs[1], cs[3]
                enum_vals[get_string(td1)] = get_string(td2)
        elif 'FIELDS' in i.attrs['id']:
            code_type = "struct"
            for row in i.find('tbody').find_all('tr'):
                cs = list(row.children)
                try:
                    td1, td2 = cs[1], cs[3]
                    cs2 = list(td2.children)
                    cs3 = list(cs2[1].children)
                    fields[get_string(td1)] = (get_string(cs3[0]), get_string(cs2[3]))
                except:
                    # title += "\n" + get_string(cs[1])
                    pass # todo comment?

    return (title, code_type, enum_vals, fields)

def get_things(sections):
    things = {}
    for child in sections:
        if child.name == 'section':
            if vals := convert(child):
                t, ct, ev, f = vals
                things[t] = (ct, ev, f)
    return things

In [183]:
def convert_type(type):
    # print(type)
    if type == 'object':
        return '[String: Any]'
    if type.startswith('object') or type.startswith('enum'):
        return type.split('(')[1][:-1]
    if type.startswith('string'):
        return 'String'
    elif type == 'integer' or type == 'number':
        return 'Int'
    elif type == 'boolean':
        return 'Bool'
    return None

def _strong_assigner(type, n='j', always=True):
    c = convert_type(type)
    if type.startswith('object'):
        return f'{c}(json: {n})'
    elif type.startswith('enum'):
        if always:
            return f'{c}(rawValue: {n} as! String)!'
        else:
            return f'{c}(rawValue: {n})!'
    elif always:
        return f'{n} as! {c}'
    else:
        return n

def _weak_type(type):
    if type.startswith('object'):
        return '[String: Any]'
    if type.startswith('enum') or type.startswith('string'):
        return 'String'
    elif type == 'integer' or type == 'number':
        return 'Int'
    elif type == 'boolean':
        return 'Bool'
    return None    

def assigner(name, type):
    if name.endswith("[]"):
        n = name[:-2]
        return f'{n} = (json["{n}"] as? [{_weak_type(type)}] ?? nil)?.map {{ j in {_strong_assigner(type)} }}\n'
    else:
        s = ''
        s += f'if let j{name} = json["{name}"] as? {_weak_type(type)} {{\n'
        s += f'\t{name} = {_strong_assigner(type, n='j'+name, always=False)}\n'
        s += '}'
        return s
        
def get_file(things):
    ss = ""
    for k, v in things.items():
        ct, ev, f = v
        s = ""
        if ct == "enum":
            s += f"enum {k}: String {{\n"
            for name, comment in ev.items():
                s += f"\tcase {name} // {comment}\n"
            s += "}"
        elif ct == "struct":
            s += f"struct {k} {{\n"
            for name, (type, comment) in f.items():
                c = convert_type(type)
                if c is None:
                    continue
                if name.endswith("[]"):
                    s += f"\tvar {name[:-2]}: [{c}]? = nil // {comment}\n"
                else:
                    s += f"\tvar {name}: {c}? = nil // {comment}\n"
            s += """
\tinit(json jsonOrNil: [String: Any]?) {
\tguard let json = jsonOrNil else {
\t\treturn
\t}
    
    """
            for name, (type, _) in f.items():
                # s += f"\t\t{assigner(name, type)}\n"
                for line in assigner(name, type).split("\n"):
                    s += f"\t\t{line}\n"
                s += "\n"
            s += "\t}\n}"
        ss += s + "\n\n"
    return ss

In [181]:
def find_section(soup):
    return [i for i in soup.find("div", attrs={'class': 'devsite-article-body'}).children if i.name == 'section'][0]

def to_swift(url, full):
    soup = bs4.BeautifulSoup(requests.get(url).text)
    s = find_section(soup)
    
    if not full:
        sections = [s]
        for i in s.children:
            if hasattr(i, 'attrs') and 'id' in i.attrs and i.attrs['id'] != i.attrs['id'].upper():
                sections.append(i)
    else:
        sections = list(s.children)
    things = get_things(sections)
    return get_file(things)

In [184]:
urls = [
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/FallbackInfo",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/LatLng",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/LocalizedText",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Location",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Money",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteModifiers",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteTravelAdvisory",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RouteTravelMode",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/RoutingPreference",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/SpeedReadingInterval",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Status",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TollInfo",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TrafficModel",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/TransitPreferences",
    "https://developers.google.com/maps/documentation/routes/reference/rest/v2/Waypoint"
]

sss = ""
sss += to_swift("https://developers.google.com/maps/documentation/routes/reference/rest/v2/TopLevel/computeRoutes", True)
for u in urls:
    sss += to_swift(u, False) + "\n"

open("test", "w").write(sss)

RouteLabel
RouteLabel
RouteLeg
RouteLeg
Int
Int
String
String
String
String
Polyline
Polyline
String
String
String
String
Viewport
Viewport
RouteTravelAdvisory
RouteTravelAdvisory
Int
Int
RouteLocalizedValues
RouteLocalizedValues
String
String
Int
Int
String
String
String
String
Polyline
Polyline
Location
Location
Location
Location
RouteLegStep
RouteLegStep
RouteLegTravelAdvisory
RouteLegTravelAdvisory
RouteLegLocalizedValues
RouteLegLocalizedValues
StepsOverview
StepsOverview
String
String
Struct format
Struct format
Int
Int
String
String
Polyline
Polyline
Location
Location
Location
Location
NavigationInstruction
NavigationInstruction
RouteLegStepTravelAdvisory
RouteLegStepTravelAdvisory
RouteLegStepLocalizedValues
RouteLegStepLocalizedValues
RouteLegStepTransitDetails
RouteLegStepTransitDetails
RouteTravelMode
RouteTravelMode
Maneuver
Maneuver
String
String
SpeedReadingInterval
SpeedReadingInterval
LocalizedText
LocalizedText
LocalizedText
LocalizedText
TransitStopDetails
TransitStop

54839